# Test Vector Leakage Assessment

Example showing usage of TVLA in this project.

In this example, there are two keys $k_a$ and $k_b$. For key $k_a$ there are $803$ (samples) available, and for key
$k_b$ there are $847$. Each sample has a length of $1400$ sample points.

In [140]:
samples_a = 803
samples_b = 847

sample_pts = 1400

The samples for these imaginary traces will be simplified into normal distributions with mean $\mu$.

A first-order "leakage" will be simulated by taking $\mu_a$ (for key $k_a$) slightly bigger than $\mu_b$.

In [141]:
mu_a = 42.2
mu_b = 42

In [142]:
import numpy as np

# Refer to section statistical properties of ASCAD in books/data/ascad_normality.
sigma = .2

traces_key_a = np.round([np.random.normal(size=samples_a, scale=sigma) + mu_a for _ in range(sample_pts)]).astype(int)
traces_key_b = np.round([np.random.normal(size=samples_b, scale=sigma) + mu_b for _ in range(sample_pts)]).astype(int)

### Step 1. Slicing the input

The first step in TVLA is to split these samples into $4$ disjoint slices of approximately equal length.

The $n$-th slice for key $k_a$ will be denoted as $A_n$.

In [143]:
from src.data.slices import tvla_slice

A = make_slice(np.moveaxis(traces_key_a, 0, -1))
B = make_slice(np.moveaxis(traces_key_b, 0, -1))

### Step 2. Oversampling

For the t-test to function properly, the slices in $A_n$ and $B_n$ should all have the same length. This is achieved by
oversampling smaller slices using SMOTE.

In [144]:
preferred_size = max([len(s) for s in [*A, *B]])

In [145]:
from src.data.smote import smote

sA = smote_slices(A, preferred_size)
sB = smote_slices(B, preferred_size)

sA.shape

(4, 212, 1400)

### Step 3. Preparing the data

In this example, the t-test is used when performing TVLA. The t-test takes the sample mean, sample variance and some
value for $p$. It tests the null hypothesis that 2 independent samples have identical average (expected) values, and
returns whether the null hypotheses should be accepted.

Therefore, $A$ and $B$ need to be transformed to fit the input parameters of the t-test.

In [146]:
mvA = np.moveaxis(np.array((sA.mean(axis=1), sA.var(axis=1))), 0, -1)
mvB = np.moveaxis(np.array((sB.mean(axis=1), sB.var(axis=1))), 0, -1)

mvA.shape

(4, 1400, 2)

### Step 4. TVLA

Create the t-test, as it relies on the number of observations (not included with the mean and variance).

In [147]:
from src.tvla.t import make_t_test

t_test = make_t_test(sample_pts)

Apply TVLA using Welch's t-test.

In [148]:
from src.tvla.main import tvla

[tvla(test=t_test, left=mvA, right=mvB, p=p) for p in range(1, 30)]

[(False, False),
 (False, False),
 (False, False),
 (False, False),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True)]

In [149]:
[tvla(test=t_test, left=mvA, right=mvA, p=p) for p in range(1, 30)]

[(False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True)]